## Data Shippers

#### Beats is the platform for single-purpose data shippers. They send data from hundreds or thousands of machines and systems to Logstash or Elasticsearch.

#### In this module we will guide the user through the installation of the beats listed below which allos the  collection of various types of host data:

* Filebeat
* Packetbeat
* Metricbeat
* Auditbeat
* Heartbeat

### Filebeat installation

In [1]:
%%bash
sudo apt install -y filebeat

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  grub-pc-bin
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  filebeat
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 25.5 MB of archives.
After this operation, 91.5 MB of additional disk space will be used.
Get:1 https://artifacts.elastic.co/packages/7.x/apt stable/main amd64 filebeat amd64 7.3.0 [25.5 MB]
Fetched 25.5 MB in 1s (27.8 MB/s)
Selecting previously unselected package filebeat.
(Reading database ... 223314 files and directories currently installed.)
Preparing to unpack .../filebeat_7.3.0_amd64.deb ...
Unpacking filebeat (7.3.0) ...
Processing triggers for ureadahead (0.100.0-21) ...
Processing triggers for systemd (237-3ubuntu10.25) ...
Setting up filebeat (7.3.0) ...
Processing triggers for systemd (237-3ubuntu10.25) ...
Processing triggers for uread

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
dpkg-preconfigure: unable to re-open stdin: No such file or directory
sh: 0: getcwd() failed: No such file or directory


### Disable default configuration

In [2]:
%%bash
sudo sed -i '28 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '117 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '148 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '150 s/^/#/' /etc/filebeat/filebeat.yml
sudo sed -i '176,178 s/^/#/' /etc/filebeat/filebeat.yml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enable Configuration to Communicate Filebeat with Logstash

##### As we want to establish a connection between filebeat and logstash, we must recall that we have secured all communication with logstash using ssl. Therefore, any beat that seeks to communicate with logstash must provide the certificate we have generated in the logstash node.

##### In this node, the certificate is located in "/etc/pki/tls/certs/logstash-forwarder.crt". Before setting up the necessary configuration for filebeat to connect with logstash, we will first validate if we can stablish a connection to logstash using the following command:

~~~
telnet <LOGSTASH-IP-ADDRESS> 5044
~~~

##### You will get a message stating "Connected to ....."  To exit press CTRL+] and then quit or CTRL+D

##### Now let us validate teh certificate by using the following command:

~~~
curl -v --cacert /etc/pki/tls/certs/logstash-forwarder.crt <LOGSTASH-IP-ADDRESS>:5044
~~~

##### Let us now proceed to set the log files we would like filebeat to track and ship to logstash, set the ip address and port in which logstash will receive our data, and define the location of our certificate that will be used to secure communication between filebeat and logstash.

In [3]:
%%bash

sudo sed -i "28i \ \ \ \ - /var/log/auth.log" /etc/filebeat/filebeat.yml
sudo sed -i "29i \ \ \ \ - /var/log/syslog" /etc/filebeat/filebeat.yml

### DO NOT EXECUTE ###
# sudo sed -i '160 s/^/#/g' /etc/filebeat/filebeat.yml 
# sudo sed -i "162i \ \ hosts: [\"$ls_ip:5044\"]" /etc/filebeat/filebeat.yml
# sudo sed -i '167i \ \ ssl.certificate_authorities: ["/etc/pki/tls/certs/logstash-forwarder.crt"]' /etc/filebeat/filebeat.yml
### DO NOT EXECUTE ###

export ls_ip="<LOGSTASH-IP-ADDRESS>"

#Enable Logstash output
sudo sed -i '/output.logstash\:/s/^#//g' /etc/filebeat/filebeat.yml

#Uncomment and Change Logstash IP ADDRESS
sudo sed -i "s/#hosts\: \[\"localhost\:5044\"\]/hosts\: \[\"$ls_ip\:5044\"\]/g" /etc/filebeat/filebeat.yml
sudo sed -i 's/#ssl\.certificate_authorities\: \["\/etc\/pki\/root\/ca\.pem"\]/ssl\.certificate\_authorities\: \["\/etc\/pki\/tls\/certs\/logstash\-forwarder\.crt"\]/g' /etc/filebeat/filebeat.yml


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enabling the Filebeat's system module.

##### Filebeat supports different modules that allows us to collect and parse logs created by Ubuntu's system loggin service. To enable the system module, run the command below:

In [4]:
%%bash
sudo filebeat modules enable system

Enabled system


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


##### You can verify if the system module has been enables by using the following command:

~~~
sudo filebeat modules list
~~~

### Loading Index templates

##### An index is a collection of documents that share similar characteristics. We will load an index template for documents stored in filebeat indexes that will be applied automatically when a new index is created. Let us list the current templates in elasticsearch by using the following command:

~~~
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template?filter_path=*.version&pretty'
~~~

In [5]:
%%bash
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template?filter_path=*.version&pretty'

{
  ".data-frame-notifications-1" : {
    "version" : 7030099
  },
  ".monitoring-kibana" : {
    "version" : 7000199
  },
  ".ml-config" : {
    "version" : 7030099
  },
  ".monitoring-logstash" : {
    "version" : 7000199
  },
  ".monitoring-es" : {
    "version" : 7000199
  },
  ".ml-anomalies-" : {
    "version" : 7030099
  },
  ".ml-meta" : {
    "version" : 7030099
  },
  ".management-beats" : {
    "version" : 70000
  },
  ".data-frame-internal-1" : {
    "version" : 7030099
  },
  ".monitoring-beats" : {
    "version" : 7000199
  },
  ".ml-notifications" : {
    "version" : 7030099
  },
  ".ml-state" : {
    "version" : 7030099
  },
  ".monitoring-alerts-7" : {
    "version" : 7000199
  },
  ".kibana_task_manager" : {
    "version" : 7030099
  }
}


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   766  100   766    0     0  63833      0 --:--:-- --:--:-- --:--:-- 63833


##### As we can observe, there are no templates defined for filebeat indexes. Let us load the filebeat template into elasticsearch by using the following command:

In [6]:
%%bash
cd ~
sudo /usr/share/filebeat/bin/filebeat export template > filebeat.template.json -c /etc/filebeat/filebeat.yml
curl -u elastic:elasticsiem -H 'Content-Type: application/json' -XPUT '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/filebeat-7.3.0' -d@/home/ubuntu/filebeat.template.json
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/' | jq 'keys'
    

{"acknowledged":true}[
  ".data-frame-internal-1",
  ".data-frame-notifications-1",
  ".kibana_task_manager",
  ".logstash-management",
  ".management-beats",
  ".ml-anomalies-",
  ".ml-config",
  ".ml-meta",
  ".ml-notifications",
  ".ml-state",
  ".monitoring-alerts-7",
  ".monitoring-beats",
  ".monitoring-es",
  ".monitoring-kibana",
  ".monitoring-logstash",
  ".triggered_watches",
  ".watch-history-10",
  ".watches",
  "filebeat-7.3.0"
]


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
chdir: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  141k  100    21  100  141k     70   471k --:--:-- --:--:-- --:--:--  473k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  106k  100  106k    0     0  4273k      0 --:--:-- --:--:-- --:--:-- 4273k


### Installing Kibana dashboards for Filebeat

In [7]:
%%bash
sudo filebeat setup -e \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=['<ELASTICSEARCH-IP-ADDRESS>:9200'] \
  -E setup.kibana.host=<KIBANA-IP-ADDRESS>:5601 \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem

Index setup finished.
Loading dashboards (Kibana must be running and reachable)
Loaded dashboards
Loaded machine learning job configurations
Loaded Ingest pipelines


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
2019-08-09T01:27:55.664Z	INFO	instance/beat.go:606	Home path: [/usr/share/filebeat] Config path: [/etc/filebeat] Data path: [/var/lib/filebeat] Logs path: [/var/log/filebeat]
2019-08-09T01:27:55.665Z	INFO	instance/beat.go:614	Beat ID: 73026322-2a2c-45b7-b2f8-3ed8088ca09c
2019-08-09T01:27:55.665Z	INFO	[beat]	instance/beat.go:902	Beat info	{"system_info": {"beat": {"path": {"config": "/etc/filebeat", "data": "/var/lib/filebeat", "home": "/usr/share/filebeat", "logs": "/var/log/filebeat"}, "type": "filebeat", "uuid": "73026322-2a2c-45b7-b2f8-3ed8088ca09c"}}}
2019-08-09T01:27:55.665Z	INFO	[beat]	instance/beat.go:911	Build info	{"system_info": {"build": {"commit": "6f0ec01a0e57fe7d4fd703b017fb5a2f6448d097", "libbeat": "7.3.0", "time": "2019-07-24T17:39:34.000Z", 

### Start Filebeat

In [34]:
%%bash
sudo systemctl start filebeat
sudo systemctl enable filebeat

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Synchronizing state of filebeat.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable filebeat
sh: 0: getcwd() failed: No such file or directory


### Alternatively, you can run Filebeat in debug mode by using the following command:

~~~
sudo /usr/share/filebeat/bin/filebeat -e -d "publish" -c /etc/filebeat/filebeat.yml
~~~

### Backloging in filebeat


##### In teh case that you change some of the filters that you set on logstash and would like to pre-process your data again, you will need to delete all indexes where your data was stored, stop filebeat, delete filebeat's registry file, and start filebeat again.

##### To delete filebeat registry, you can use the following command:

~~~
sudo rm -R /var/lib/filebeat/registry
~~~

##### Note: Whenever you process again your data, your receiving logs will have a timestamp stating the time the log was received by elasticsearch and not the time at which the log was generated. The timestamp can be replaced by adding a rule on logstash that takes the timestamp stated at the beggining of each log and replace teh default timestamp (receiving time) set by elasticsearch.


### Packetbeat Installation

In [9]:
%%bash
sudo apt install -y packetbeat

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  grub-pc-bin
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  packetbeat
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 21.3 MB of archives.
After this operation, 76.0 MB of additional disk space will be used.
Get:1 https://artifacts.elastic.co/packages/7.x/apt stable/main amd64 packetbeat amd64 7.3.0 [21.3 MB]
Fetched 21.3 MB in 2s (13.1 MB/s)
Selecting previously unselected package packetbeat.
(Reading database ... 223782 files and directories currently installed.)
Preparing to unpack .../packetbeat_7.3.0_amd64.deb ...
Unpacking packetbeat (7.3.0) ...
Setting up packetbeat (7.3.0) ...
Processing triggers for ureadahead (0.100.0-21) ...
Processing triggers for systemd (237-3ubuntu10.25) ...


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
dpkg-preconfigure: unable to re-open stdin: No such file or directory
sh: 0: getcwd() failed: No such file or directory


### Disable default configuration

##### The default packetbeat configuration enables monitoring certain transaction porotocols such as icmp, ampq, cassandra, among others. We will disable unwanted protocols by commenting the protocol and list of ports and monitor only the following: icmp, dhcpv4, dns, http, and tls.

In [10]:
%%bash
sudo sed -i 's/^\- type\: amqp/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[5672\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: cassandra/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[9042\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: memcache/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[11211\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: mysql/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[3306,3307\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: pgsql/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[5432\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: redis/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[6379\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: thrift/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[9090\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: mongodb/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[27017\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\- type\: nfs/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ ports\: \[2049\]/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^output.elasticsearch\:/#&/' /etc/packetbeat/packetbeat.yml
sudo sed -i 's/^\ \ hosts\: \[\"localhost\:9200\"\]/#&/' /etc/packetbeat/packetbeat.yml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enable communication with logstash

In [11]:
%%bash
export ls_ip="<LOGSTASH-IP-ADDRESS>"

#Enable Logstash output
sudo sed -i '/output.logstash\:/s/^#//g' /etc/packetbeat/packetbeat.yml

#Uncomment and Change Logstash IP ADDRESS
sudo sed -i "s/#hosts\: \[\"localhost\:5044\"\]/hosts\: \[\"$ls_ip\:5044\"\]/g" /etc/packetbeat/packetbeat.yml
sudo sed -i 's/#ssl\.certificate_authorities\: \["\/etc\/pki\/root\/ca\.pem"\]/ssl\.certificate\_authorities\: \["\/etc\/pki\/tls\/certs\/logstash\-forwarder\.crt"\]/g' /etc/packetbeat/packetbeat.yml


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Test Configuration and Connectivity to Logstash

In [17]:
%%bash
sudo /usr/share/packetbeat/bin/packetbeat test config -c /etc/packetbeat/packetbeat.yml
sudo /usr/share/packetbeat/bin/packetbeat test output -c /etc/packetbeat/packetbeat.yml

Config OK
logstash: 10.1.1.21:5044...
  connection...
    parse host... OK
    dns lookup... OK
    addresses: 10.1.1.21
    dial up... OK
  TLS...
    security: server's certificate chain verification is enabled
    handshake... OK
    TLS version: TLSv1.2
    dial up... OK
  talk to server... OK


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Add Kibana Dashboards for Packetbeat

In [18]:
%%bash
sudo packetbeat setup -e \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=['<ELASTICSEARCH-IP-ADDRESS>:9200'] \
  -E setup.kibana.host=<KIBANA-IP-ADDRESS>:5601 \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem

Index setup finished.
Loading dashboards (Kibana must be running and reachable)
Loaded dashboards


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
2019-08-09T01:40:37.171Z	INFO	instance/beat.go:606	Home path: [/usr/share/packetbeat] Config path: [/etc/packetbeat] Data path: [/var/lib/packetbeat] Logs path: [/var/log/packetbeat]
2019-08-09T01:40:37.882Z	INFO	instance/beat.go:614	Beat ID: 7941b4bc-3e6f-4b50-9800-43bd7191ec37
2019-08-09T01:40:37.883Z	INFO	[beat]	instance/beat.go:902	Beat info	{"system_info": {"beat": {"path": {"config": "/etc/packetbeat", "data": "/var/lib/packetbeat", "home": "/usr/share/packetbeat", "logs": "/var/log/packetbeat"}, "type": "packetbeat", "uuid": "7941b4bc-3e6f-4b50-9800-43bd7191ec37"}}}
2019-08-09T01:40:37.884Z	INFO	[beat]	instance/beat.go:911	Build info	{"system_info": {"build": {"commit": "6f0ec01a0e57fe7d4fd703b017fb5a2f6448d097", "libbeat": "7.3.0", "time": "2019-07-2

### Add template from packetbeat to elasticsearch

In [19]:
%%bash

sudo packetbeat setup --index-management \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=["<ELASTICSEARCH-IP-ADDRESS>:9200"] \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem


Index setup finished.


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### List Loaded Templates

In [20]:
%%bash
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/' | jq 'keys'

[
  ".data-frame-internal-1",
  ".data-frame-notifications-1",
  ".kibana_task_manager",
  ".logstash-management",
  ".management-beats",
  ".ml-anomalies-",
  ".ml-config",
  ".ml-meta",
  ".ml-notifications",
  ".ml-state",
  ".monitoring-alerts-7",
  ".monitoring-beats",
  ".monitoring-es",
  ".monitoring-kibana",
  ".monitoring-logstash",
  ".triggered_watches",
  ".watch-history-10",
  ".watches",
  "filebeat-7.3.0",
  "packetbeat-7.3.0"
]


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  151k  100  151k    0     0  5827k      0 --:--:-- --:--:-- --:--:-- 5827k


### Start Packetbeat

In [21]:
%%bash
sudo systemctl start packetbeat
sudo systemctl enable packetbeat

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Synchronizing state of packetbeat.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable packetbeat
sh: 0: getcwd() failed: No such file or directory


### Alternatively, you can run Filebeat in debug mode by using the following command:

~~~
sudo /usr/share/packetbeat/bin/packetbeat -e -d "publish" -c /etc/packetbeat/packetbeat.yml
~~~

### Metricbeat Installation

In [22]:
%%bash
sudo apt install -y metricbeat

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  grub-pc-bin
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  metricbeat
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 40.3 MB of archives.
After this operation, 176 MB of additional disk space will be used.
Get:1 https://artifacts.elastic.co/packages/7.x/apt stable/main amd64 metricbeat amd64 7.3.0 [40.3 MB]
Fetched 40.3 MB in 2s (21.4 MB/s)
Selecting previously unselected package metricbeat.
(Reading database ... 223815 files and directories currently installed.)
Preparing to unpack .../metricbeat_7.3.0_amd64.deb ...
Unpacking metricbeat (7.3.0) ...
Setting up metricbeat (7.3.0) ...
Processing triggers for ureadahead (0.100.0-21) ...
Processing triggers for systemd (237-3ubuntu10.25) ...


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
dpkg-preconfigure: unable to re-open stdin: No such file or directory
sh: 0: getcwd() failed: No such file or directory


### Disable default configuration

##### The default metricbeat configuration enables connectivity directly to elasticseach. We will disable this configuration to later enable communication to logstash.

In [23]:
%%bash
sudo sed -i 's/^output.elasticsearch\:/#&/' /etc/metricbeat/metricbeat.yml
sudo sed -i 's/^\ \ hosts\: \[\"localhost\:9200\"\]/#&/' /etc/metricbeat/metricbeat.yml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enable communication with logstash

In [24]:
%%bash
export ls_ip="<LOGSTASH-IP-ADDRESS>"

#Enable Logstash output
sudo sed -i '/output.logstash\:/s/^#//g' /etc/metricbeat/metricbeat.yml

#Uncomment and Change Logstash IP ADDRESS
sudo sed -i "s/#hosts\: \[\"localhost\:5044\"\]/hosts\: \[\"$ls_ip\:5044\"\]/g" /etc/metricbeat/metricbeat.yml
sudo sed -i 's/#ssl\.certificate_authorities\: \["\/etc\/pki\/root\/ca\.pem"\]/ssl\.certificate\_authorities\: \["\/etc\/pki\/tls\/certs\/logstash\-forwarder\.crt"\]/g' /etc/metricbeat/metricbeat.yml


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enabling the Metricbeats's system module.

##### Metric supports different modules that allows us to collect Metrics from different services. In our case, we would like to enable the System Metric module to monitor the metrics of our Virtual Machine.

##### For more infomration about the available modules check the following link:
https://www.elastic.co/guide/en/beats/metricbeat/current/metricbeat-modules.html

##### To enable the system module, run the command below:

In [25]:
%%bash
sudo metricbeat modules enable system

Module system is already enabled


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


##### You can verify if the system module has been enables by using the following command:

~~~
sudo metricbeat modules list
~~~

### Test Configuration and Connectivity to Logstash

In [26]:
%%bash
sudo /usr/share/metricbeat/bin/metricbeat test config -c /etc/metricbeat/metricbeat.yml
sudo /usr/share/metricbeat/bin/metricbeat test output -c /etc/metricbeat/metricbeat.yml

Config OK
logstash: 10.1.1.21:5044...
  connection...
    parse host... OK
    dns lookup... OK
    addresses: 10.1.1.21
    dial up... OK
  TLS...
    security: server's certificate chain verification is enabled
    handshake... OK
    TLS version: TLSv1.2
    dial up... OK
  talk to server... OK


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Add Kibana Dashboards for Metricbeat

In [27]:
%%bash
sudo metricbeat setup -e \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=['<ELASTICSEARCH-IP-ADDRESS>:9200'] \
  -E setup.kibana.host=<KIBANA-IP-ADDRESS>:5601 \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem

Index setup finished.
Loading dashboards (Kibana must be running and reachable)
Loaded dashboards


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
2019-08-09T01:44:46.234Z	INFO	instance/beat.go:606	Home path: [/usr/share/metricbeat] Config path: [/etc/metricbeat] Data path: [/var/lib/metricbeat] Logs path: [/var/log/metricbeat]
2019-08-09T01:44:46.234Z	INFO	instance/beat.go:614	Beat ID: 6d819694-245a-4eb0-a06e-0323b9aa58ee
2019-08-09T01:44:46.235Z	INFO	[beat]	instance/beat.go:902	Beat info	{"system_info": {"beat": {"path": {"config": "/etc/metricbeat", "data": "/var/lib/metricbeat", "home": "/usr/share/metricbeat", "logs": "/var/log/metricbeat"}, "type": "metricbeat", "uuid": "6d819694-245a-4eb0-a06e-0323b9aa58ee"}}}
2019-08-09T01:44:46.235Z	INFO	[beat]	instance/beat.go:911	Build info	{"system_info": {"build": {"commit": "6f0ec01a0e57fe7d4fd703b017fb5a2f6448d097", "libbeat": "7.3.0", "time": "2019-07-2

### Add template from metricbeat to elasticsearch

In [28]:
%%bash

sudo metricbeat setup --index-management \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=["<ELASTICSEARCH-IP-ADDRESS>:9200"] \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem


Index setup finished.


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### List Loaded Templates

In [29]:
%%bash
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/' | jq 'keys'

[
  ".data-frame-internal-1",
  ".data-frame-notifications-1",
  ".kibana_task_manager",
  ".logstash-management",
  ".management-beats",
  ".ml-anomalies-",
  ".ml-config",
  ".ml-meta",
  ".ml-notifications",
  ".ml-state",
  ".monitoring-alerts-7",
  ".monitoring-beats",
  ".monitoring-es",
  ".monitoring-kibana",
  ".monitoring-logstash",
  ".triggered_watches",
  ".watch-history-10",
  ".watches",
  "filebeat-7.3.0",
  "metricbeat-7.3.0",
  "packetbeat-7.3.0"
]


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  286k  100  286k    0     0  6237k      0 --:--:-- --:--:-- --:--:-- 6237k


### Start Metricbeat

In [30]:
%%bash
sudo systemctl start metricbeat
sudo systemctl enable metricbeat

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Synchronizing state of metricbeat.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable metricbeat
sh: 0: getcwd() failed: No such file or directory


### Alternatively, you can run Metricbeat in debug mode by using the following command:

~~~
sudo /usr/share/metriceat/bin/metricbeat -e -d "publish" -c /etc/metricbeat/metricbeat.yml
~~~

### Auditbeat Installation

In [31]:
%%bash
sudo apt install -y auditbeat

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  grub-pc-bin
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  auditbeat
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 20.3 MB of archives.
After this operation, 72.9 MB of additional disk space will be used.
Get:1 https://artifacts.elastic.co/packages/7.x/apt stable/main amd64 auditbeat amd64 7.3.0 [20.3 MB]
Fetched 20.3 MB in 2s (12.2 MB/s)
Selecting previously unselected package auditbeat.
(Reading database ... 223929 files and directories currently installed.)
Preparing to unpack .../auditbeat_7.3.0_amd64.deb ...
Unpacking auditbeat (7.3.0) ...
Setting up auditbeat (7.3.0) ...
Processing triggers for ureadahead (0.100.0-21) ...
Processing triggers for systemd (237-3ubuntu10.25) ...


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
dpkg-preconfigure: unable to re-open stdin: No such file or directory
sh: 0: getcwd() failed: No such file or directory


### Disable default configuration

##### The default metricbeat configuration enables connectivity directly to elasticseach. We will disable this configuration to later enable communication to logstash.

In [32]:
%%bash
sudo sed -i 's/^output.elasticsearch\:/#&/' /etc/auditbeat/auditbeat.yml
sudo sed -i 's/^\ \ hosts\: \[\"localhost\:9200\"\]/#&/' /etc/auditbeat/auditbeat.yml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enable communication with logstash

In [33]:
%%bash
export ls_ip="<LOGSTASH-IP-ADDRESS>"

#Enable Logstash output
sudo sed -i '/output.logstash\:/s/^#//g' /etc/auditbeat/auditbeat.yml

#Uncomment and Change Logstash IP ADDRESS
sudo sed -i "s/#hosts\: \[\"localhost\:5044\"\]/hosts\: \[\"$ls_ip\:5044\"\]/g" /etc/auditbeat/auditbeat.yml
sudo sed -i 's/#ssl\.certificate_authorities\: \["\/etc\/pki\/root\/ca\.pem"\]/ssl\.certificate\_authorities\: \["\/etc\/pki\/tls\/certs\/logstash\-forwarder\.crt"\]/g' /etc/auditbeat/auditbeat.yml


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Change timefram to obtain and update on the system's state

In [35]:
%%bash
sudo sed -i "s/state\.period\: 12h/state\.period\: 1h/g" /etc/auditbeat/auditbeat.yml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enabling the Auditbeats's system module.

##### In the configuration file, two modules are configured by default 1) Auditd 2) File_Integrity and 3) System with the details presented below.
##### As you can see, there are no rules defined for auditd. In addition, auditd is not installed in our system.

~~~
- module: auditd
  # Load audit rules from separate files. Same format as audit.rules(7).
  audit_rule_files: [ '${path.config}/audit.rules.d/*.conf' ]
  audit_rules: |
  
- module: file_integrity
  paths:
  - /bin
  - /usr/bin
  - /sbin
  - /usr/sbin
  - /etc
  
- module: system
  datasets:
    - host    # General host information, e.g. uptime, IPs
    - process # Started and stopped processes
    - socket  # Opened and closed sockets
    - user    # User information
~~~

##### You can check available modules in teh link below if you seed to audit additional data:
https://www.elastic.co/guide/en/beats/auditbeat/current/auditbeat-modules.html

##### We proceed to install auditd on Ubuntu 18.04 and to enable module in auditbeat.

### Install auditd in ubuntu 18.04:

In [36]:
%%bash
sudo apt install -y auditd audispd-plugins

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  grub-pc-bin
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-lato javascript-common libauparse0 libjs-jquery libltdl7 libprelude23
  libruby2.5 prelude-utils rake ruby ruby-did-you-mean ruby-minitest
  ruby-net-telnet ruby-power-assert ruby-test-unit ruby2.5
  rubygems-integration unzip zip
Suggested packages:
  ri ruby-dev bundler
The following NEW packages will be installed:
  audispd-plugins auditd fonts-lato javascript-common libauparse0 libjs-jquery
  libltdl7 libprelude23 libruby2.5 prelude-utils rake ruby ruby-did-you-mean
  ruby-minitest ruby-net-telnet ruby-power-assert ruby-test-unit ruby2.5
  rubygems-integration unzip zip
0 upgraded, 21 newly installed, 0 to remove and 0 not upgraded.
Need to get 7096 kB of archives.
After this operation, 32.5 MB of additional disk

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
dpkg-preconfigure: unable to re-open stdin: No such file or directory
sh: 0: getcwd() failed: No such file or directory


### Create rules for Auditd

##### For reference information for each rule please visit the following site:

https://gist.githubusercontent.com/Neo23x0/9fe88c0c5979e017a389b90fd19ddfee/raw/f37173a62e2d54325b0e5d496ac51c37dfb4e37f/audit.rules

In [37]:
%%bash
# Remove any existing rules
sudo auditctl -D

# Buffer Size
## Feel free to increase this if the machine panic's
sudo auditctl -b 8192

# Failure Mode
## Possible values: 0 (silent), 1 (printk, print a failure message), 2 (panic, halt the system)
sudo auditctl -f 1

# Ignore errors
## e.g. caused by users or files not found in the local environment  
sudo auditctl -i 

# Self Auditing ---------------------------------------------------------------

## Audit the audit logs
### Successful and unsuccessful attempts to read information from the audit records
sudo auditctl -w /var/log/audit/ -k auditlog

## Auditd configuration
### Modifications to audit configuration that occur while the audit collection functions are operating
sudo auditctl -w /etc/audit/ -p wa -k auditconfig
sudo auditctl -w /etc/libaudit.conf -p wa -k auditconfig
sudo auditctl -w /etc/audisp/ -p wa -k audispconfig

## Monitor for use of audit management tools
sudo auditctl -w /sbin/auditctl -p x -k audittools
sudo auditctl -w /sbin/auditd -p x -k audittools

# Filters ---------------------------------------------------------------------

### We put these early because audit is a first match wins system.

## Ignore SELinux AVC records
sudo auditctl -a always,exclude -F msgtype=AVC

## Ignore current working directory records
sudo auditctl -a always,exclude -F msgtype=CWD

## Ignore EOE records (End Of Event, not needed)
sudo auditctl -a always,exclude -F msgtype=EOE

## Cron jobs fill the logs with stuff we normally don't want (works with SELinux)
sudo auditctl -a never,user -F subj_type=crond_t
sudo auditctl -a exit,never -F subj_type=crond_t

## This prevents chrony from overwhelming the logs
sudo auditctl -a never,exit -F arch=b64 -S adjtimex -F auid=unset -F uid=chrony -F subj_type=chronyd_t

## This is not very interesting and wastes a lot of space if the server is public facing
sudo auditctl -a always,exclude -F msgtype=CRYPTO_KEY_USER

## VMWare tools
sudo auditctl -a exit,never -F arch=b32 -S fork -F success=0 -F path=/usr/lib/vmware-tools -F subj_type=initrc_t -F exit=-2
sudo auditctl -a exit,never -F arch=b64 -S fork -F success=0 -F path=/usr/lib/vmware-tools -F subj_type=initrc_t -F exit=-2

### High Volume Event Filter (especially on Linux Workstations)
sudo auditctl -a exit,never -F arch=b32 -F dir=/dev/shm -k sharedmemaccess
sudo auditctl -a exit,never -F arch=b64 -F dir=/dev/shm -k sharedmemaccess
sudo auditctl -a exit,never -F arch=b32 -F dir=/var/lock/lvm -k locklvm
sudo auditctl -a exit,never -F arch=b64 -F dir=/var/lock/lvm -k locklvm

## More information on how to filter events
### https://access.redhat.com/solutions/2482221

# Rules -----------------------------------------------------------------------

## Kernel parameters
sudo auditctl -w /etc/sysctl.conf -p wa -k sysctl

## Kernel module loading and unloading
sudo auditctl -a always,exit -F perm=x -F auid!=-1 -F path=/sbin/insmod -k modules
sudo auditctl -a always,exit -F perm=x -F auid!=-1 -F path=/sbin/modprobe -k modules
sudo auditctl -a always,exit -F perm=x -F auid!=-1 -F path=/sbin/rmmod -k modules
sudo auditctl -a always,exit -F arch=b64 -S finit_module -S init_module -S delete_module -F auid!=-1 -k modules
sudo auditctl -a always,exit -F arch=b32 -S finit_module -S init_module -S delete_module -F auid!=-1 -k modules
## Modprobe configuration
sudo auditctl -w /etc/modprobe.conf -p wa -k modprobe

## KExec usage (all actions)
sudo auditctl -a always,exit -F arch=b64 -S kexec_load -k KEXEC
sudo auditctl -a always,exit -F arch=b32 -S sys_kexec_load -k KEXEC

## Special files
sudo auditctl -a exit,always -F arch=b32 -S mknod -S mknodat -k specialfiles
sudo auditctl -a exit,always -F arch=b64 -S mknod -S mknodat -k specialfiles

## Mount operations (only attributable)
sudo auditctl -a always,exit -F arch=b64 -S mount -S umount2 -F auid!=-1 -k mount
sudo auditctl -a always,exit -F arch=b32 -S mount -S umount -S umount2 -F auid!=-1 -k mount

# Change swap (only attributable)
sudo auditctl -a always,exit -F arch=b64 -S swapon -S swapoff -F auid!=-1 -k swap
sudo auditctl -a always,exit -F arch=b32 -S swapon -S swapoff -F auid!=-1 -k swap

## Time
sudo auditctl -a exit,always -F arch=b32 -S adjtimex -S settimeofday -S clock_settime -k time
sudo auditctl -a exit,always -F arch=b64 -S adjtimex -S settimeofday -S clock_settime -k time
### Local time zone
sudo auditctl -w /etc/localtime -p wa -k localtime

## Stunnel
sudo auditctl -w /usr/sbin/stunnel -p x -k stunnel

## Cron configuration & scheduled jobs
sudo auditctl -w /etc/cron.allow -p wa -k cron
sudo auditctl -w /etc/cron.deny -p wa -k cron
sudo auditctl -w /etc/cron.d/ -p wa -k cron
sudo auditctl -w /etc/cron.daily/ -p wa -k cron
sudo auditctl -w /etc/cron.hourly/ -p wa -k cron
sudo auditctl -w /etc/cron.monthly/ -p wa -k cron
sudo auditctl -w /etc/cron.weekly/ -p wa -k cron
sudo auditctl -w /etc/crontab -p wa -k cron
sudo auditctl -w /var/spool/cron/crontabs/ -k cron

## User, group, password databases
sudo auditctl -w /etc/group -p wa -k etcgroup
sudo auditctl -w /etc/passwd -p wa -k etcpasswd
sudo auditctl -w /etc/gshadow -k etcgroup
sudo auditctl -w /etc/shadow -k etcpasswd
sudo auditctl -w /etc/security/opasswd -k opasswd

## Sudoers file changes
sudo auditctl -w /etc/sudoers -p wa -k actions

## Passwd
sudo auditctl -w /usr/bin/passwd -p x -k passwd_modification

## Tools to change group identifiers
sudo auditctl -w /usr/sbin/groupadd -p x -k group_modification
sudo auditctl -w /usr/sbin/groupmod -p x -k group_modification
sudo auditctl -w /usr/sbin/addgroup -p x -k group_modification
sudo auditctl -w /usr/sbin/useradd -p x -k user_modification
sudo auditctl -w /usr/sbin/usermod -p x -k user_modification
sudo auditctl -w /usr/sbin/adduser -p x -k user_modification

## Login configuration and information
sudo auditctl -w /etc/login.defs -p wa -k login
sudo auditctl -w /etc/securetty -p wa -k login
sudo auditctl -w /var/log/faillog -p wa -k login
sudo auditctl -w /var/log/lastlog -p wa -k login
sudo auditctl -w /var/log/tallylog -p wa -k login

## Network Environment
### Changes to hostname
sudo auditctl -a always,exit -F arch=b32 -S sethostname -S setdomainname -k network_modifications
sudo auditctl -a always,exit -F arch=b64 -S sethostname -S setdomainname -k network_modifications
### Changes to other files
sudo auditctl -w /etc/hosts -p wa -k network_modifications
sudo auditctl -w /etc/sysconfig/network -p wa -k network_modifications
sudo auditctl -w /etc/network/ -p wa -k network
sudo auditctl -a always,exit -F dir=/etc/NetworkManager/ -F perm=wa -k network_modifications
sudo auditctl -w /etc/sysconfig/network -p wa -k network_modifications
### Changes to issue
sudo auditctl -w /etc/issue -p wa -k etcissue
sudo auditctl -w /etc/issue.net -p wa -k etcissue

## System startup scripts
sudo auditctl -w /etc/inittab -p wa -k init
sudo auditctl -w /etc/init.d/ -p wa -k init
sudo auditctl -w /etc/init/ -p wa -k init

## Library search paths
sudo auditctl -w /etc/ld.so.conf -p wa -k libpath

## Pam configuration
sudo auditctl -w /etc/pam.d/ -p wa -k pam
sudo auditctl -w /etc/security/limits.conf -p wa  -k pam
sudo auditctl -w /etc/security/pam_env.conf -p wa -k pam
sudo auditctl -w /etc/security/namespace.conf -p wa -k pam
sudo auditctl -w /etc/security/namespace.init -p wa -k pam

## Postfix configuration
sudo auditctl -w /etc/aliases -p wa -k mail
sudo auditctl -w /etc/postfix/ -p wa -k mail

## SSH configuration
sudo auditctl -w /etc/ssh/sshd_config -k sshd

# Systemd
sudo auditctl -w /bin/systemctl -p x -k systemd 
sudo auditctl -w /etc/systemd/ -p wa -k systemd

## SELinux events that modify the system's Mandatory Access Controls (MAC)
sudo auditctl -w /etc/selinux/ -p wa -k mac_policy

## Critical elements access failures 
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/etc -F success=0 -k unauthedfileaccess
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/bin -F success=0 -k unauthedfileaccess
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/sbin -F success=0 -k unauthedfileaccess
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/usr/bin -F success=0 -k unauthedfileaccess
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/usr/sbin -F success=0 -k unauthedfileaccess
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/var -F success=0 -k unauthedfileaccess
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/home -F success=0 -k unauthedfileaccess
sudo auditctl -a exit,always -F arch=b64 -S open -F dir=/srv -F success=0 -k unauthedfileaccess

## Process ID change (switching accounts) applications
sudo auditctl -w /bin/su -p x -k priv_esc
sudo auditctl -w /usr/bin/sudo -p x -k priv_esc
sudo auditctl -w /etc/sudoers -p rw -k priv_esc

## Power state
sudo auditctl -w /sbin/shutdown -p x -k power
sudo auditctl -w /sbin/poweroff -p x -k power
sudo auditctl -w /sbin/reboot -p x -k power
sudo auditctl -w /sbin/halt -p x -k power

## Session initiation information
sudo auditctl -w /var/run/utmp -p wa -k session
sudo auditctl -w /var/log/btmp -p wa -k session
sudo auditctl -w /var/log/wtmp -p wa -k session

## Discretionary Access Control (DAC) modifications
sudo auditctl -a always,exit -F arch=b32 -S chmod -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S chown -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S fchmod -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S fchmodat -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S fchown -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S fchownat -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S fremovexattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S fsetxattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S lchown -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S lremovexattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S lsetxattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S removexattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b32 -S setxattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S chmod  -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S chown -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S fchmod -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S fchmodat -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S fchown -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S fchownat -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S fremovexattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S fsetxattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S lchown -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S lremovexattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S lsetxattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S removexattr -F auid>=500 -F auid!=4294967295 -k perm_mod
sudo auditctl -a always,exit -F arch=b64 -S setxattr -F auid>=500 -F auid!=4294967295 -k perm_mod

# Special Rules ---------------------------------------------------------------

## 32bit API Exploitation
### If you are on a 64 bit platform, everything _should_ be running
### in 64 bit mode. This rule will detect any use of the 32 bit syscalls
### because this might be a sign of someone exploiting a hole in the 32
### bit API.
sudo auditctl -a always,exit -F arch=b32 -S all -k 32bit_api

## Reconnaissance
sudo auditctl -w /usr/bin/whoami -p x -k recon
sudo auditctl -w /etc/issue -p r -k recon
sudo auditctl -w /etc/hostname -p r -k recon

## Suspicious activity
sudo auditctl -w /usr/bin/wget -p x -k susp_activity
sudo auditctl -w /usr/bin/curl -p x -k susp_activity
sudo auditctl -w /usr/bin/base64 -p x -k susp_activity
sudo auditctl -w /bin/nc -p x -k susp_activity
sudo auditctl -w /bin/netcat -p x -k susp_activity
sudo auditctl -w /usr/bin/ncat -p x -k susp_activity
sudo auditctl -w /usr/bin/ssh -p x -k susp_activity
sudo auditctl -w /usr/bin/socat -p x -k susp_activity
sudo auditctl -w /usr/bin/wireshark -p x -k susp_activity
sudo auditctl -w /usr/bin/rawshark -p x -k susp_activity
sudo auditctl -w /usr/bin/rdesktop -p x -k sbin_susp

## Sbin suspicious activity
sudo auditctl -w /sbin/iptables -p x -k sbin_susp 
sudo auditctl -w /sbin/ifconfig -p x -k sbin_susp
sudo auditctl -w /usr/sbin/tcpdump -p x -k sbin_susp
sudo auditctl -w /usr/sbin/traceroute -p x -k sbin_susp

## Injection 
### These rules watch for code injection by the ptrace facility.
### This could indicate someone trying to do something bad or just debugging
sudo auditctl -a always,exit -F arch=b32 -S ptrace -k tracing
sudo auditctl -a always,exit -F arch=b64 -S ptrace -k tracing
sudo auditctl -a always,exit -F arch=b32 -S ptrace -F a0=0x4 -k code_injection
sudo auditctl -a always,exit -F arch=b64 -S ptrace -F a0=0x4 -k code_injection
sudo auditctl -a always,exit -F arch=b32 -S ptrace -F a0=0x5 -k data_injection
sudo auditctl -a always,exit -F arch=b64 -S ptrace -F a0=0x5 -k data_injection
sudo auditctl -a always,exit -F arch=b32 -S ptrace -F a0=0x6 -k register_injection
sudo auditctl -a always,exit -F arch=b64 -S ptrace -F a0=0x6 -k register_injection

## Privilege Abuse
### The purpose of this rule is to detect when an admin may be abusing power by looking in user's home dir.
sudo auditctl -a always,exit -F dir=/home -F uid=0 -F auid>=1000 -F auid!=4294967295 -C auid!=obj_uid -k power_abuse

# Software Management ---------------------------------------------------------

# DPKG / APT-GET (Debian/Ubuntu)
sudo auditctl -w /usr/bin/dpkg -p x -k software_mgmt
sudo auditctl -w /usr/bin/apt-add-repository -p x -k software_mgmt
sudo auditctl -w /usr/bin/apt-get -p x -k software_mgmt
sudo auditctl -w /usr/bin/aptitude -p x -k software_mgmt

# High volume events ----------------------------------------------------------

## Remove them if the cause to much volumen in your einvironment

## Root command executions 
sudo auditctl -a exit,always -F arch=b64 -F euid=0 -S execve -k rootcmd
sudo auditctl -a exit,always -F arch=b32 -F euid=0 -S execve -k rootcmd

## File Deletion Events by User
sudo auditctl -a always,exit -F arch=b32 -S rmdir -S unlink -S unlinkat -S rename -S renameat -F auid>=500 -F auid!=4294967295 -k delete
sudo auditctl -a always,exit -F arch=b64 -S rmdir -S unlink -S unlinkat -S rename -S renameat -F auid>=500 -F auid!=4294967295 -k delete

## File Access
### Unauthorized Access (unsuccessful)
sudo auditctl -a always,exit -F arch=b32 -S creat -S open -S openat -S open_by_handle_at -S truncate -S ftruncate -F exit=-EACCES -F auid>=500 -F auid!=4294967295 -k file_access
sudo auditctl -a always,exit -F arch=b32 -S creat -S open -S openat -S open_by_handle_at -S truncate -S ftruncate -F exit=-EPERM -F auid>=500 -F auid!=4294967295 -k file_access
sudo auditctl -a always,exit -F arch=b64 -S creat -S open -S openat -S open_by_handle_at -S truncate -S ftruncate -F exit=-EACCES -F auid>=500 -F auid!=4294967295 -k file_access
sudo auditctl -a always,exit -F arch=b64 -S creat -S open -S openat -S open_by_handle_at -S truncate -S ftruncate -F exit=-EPERM -F auid>=500 -F auid!=4294967295 -k file_access

### Unsuccessful Creation
sudo auditctl -a always,exit -F arch=b32 -S creat,link,mknod,mkdir,symlink,mknodat,linkat,symlinkat -F exit=-EACCES -k file_creation
sudo auditctl -a always,exit -F arch=b64 -S mkdir,creat,link,symlink,mknod,mknodat,linkat,symlinkat -F exit=-EACCES -k file_creation
sudo auditctl -a always,exit -F arch=b32 -S link,mkdir,symlink,mkdirat -F exit=-EPERM -k file_creation
sudo auditctl -a always,exit -F arch=b64 -S mkdir,link,symlink,mkdirat -F exit=-EPERM -k file_creation

### Unsuccessful Modification
sudo auditctl -a always,exit -F arch=b32 -S rename -S renameat -S truncate -S chmod -S setxattr -S lsetxattr -S removexattr -S lremovexattr -F exit=-EACCES -k file_modification
sudo auditctl -a always,exit -F arch=b64 -S rename -S renameat -S truncate -S chmod -S setxattr -S lsetxattr -S removexattr -S lremovexattr -F exit=-EACCES -k file_modification
sudo auditctl -a always,exit -F arch=b32 -S rename -S renameat -S truncate -S chmod -S setxattr -S lsetxattr -S removexattr -S lremovexattr -F exit=-EPERM -k file_modification
sudo auditctl -a always,exit -F arch=b64 -S rename -S renameat -S truncate -S chmod -S setxattr -S lsetxattr -S removexattr -S lremovexattr -F exit=-EPERM -k file_modification

# Make the configuration immutable --------------------------------------------
##-e 2

No rules
enabled 1
failure 1
pid 28162
rate_limit 0
backlog_limit 8192
lost 0
backlog 0
backlog_wait_time 0
enabled 1
failure 1
pid 28162
rate_limit 0
backlog_limit 8192
lost 0
backlog 1
backlog_wait_time 0


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Unknown user: chrony
-F unknown field: uid
Error sending add rule data request (No such file or directory)
Error sending add rule data request (No such file or directory)
bash: line 216: =500: No such file or directory
bash: line 217: =500: No such file or directory
bash: line 218: =500: No such file or directory
bash: line 219: =500: No such file or directory
bash: line 220: =500: No such file or directory
bash: line 221: =500: No such file or directory
bash: line 222: =500: No such file or directory
bash: line 223: =500: No such file or directory
bash: line 224: =500: No such file or directory
bash: line 225: =500: No such file or directory
bash: line 226: =500: No such file or directory
bash: line 227: =500: No such file or directory
bash: line 228: =500: No such file or directory
bash: line 229: =500: No such file or directory
bash: line 230: =500: No such file or dir

### Disable jornald from listening to audit messages

In [39]:
%%bash
sudo systemctl mask systemd-journald-audit.socket

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Created symlink /etc/systemd/system/systemd-journald-audit.socket → /dev/null.


### Test Configuration and Connectivity to Logstash

In [40]:
%%bash
sudo /usr/share/auditbeat/bin/auditbeat test config -c /etc/auditbeat/auditbeat.yml
sudo /usr/share/auditbeat/bin/auditbeat test output -c /etc/auditbeat/auditbeat.yml

Config OK
logstash: 10.1.1.21:5044...
  connection...
    parse host... OK
    dns lookup... OK
    addresses: 10.1.1.21
    dial up... OK
  TLS...
    security: server's certificate chain verification is enabled
    handshake... OK
    TLS version: TLSv1.2
    dial up... OK
  talk to server... OK


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Add Kibana Dashboards for Auditbeat

In [41]:
%%bash
sudo auditbeat setup -e \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=['<ELASTICSEARCH-IP-ADDRESS>:9200'] \
  -E setup.kibana.host=<KIBANA-IP-ADDRESS>:5601 \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem

Index setup finished.
Loading dashboards (Kibana must be running and reachable)
Loaded dashboards


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
2019-08-09T02:04:41.363Z	INFO	instance/beat.go:606	Home path: [/usr/share/auditbeat] Config path: [/etc/auditbeat] Data path: [/var/lib/auditbeat] Logs path: [/var/log/auditbeat]
2019-08-09T02:04:41.388Z	INFO	instance/beat.go:614	Beat ID: 67b75f47-b666-4b1c-beae-dde90079e8f1
2019-08-09T02:04:41.390Z	INFO	[beat]	instance/beat.go:902	Beat info	{"system_info": {"beat": {"path": {"config": "/etc/auditbeat", "data": "/var/lib/auditbeat", "home": "/usr/share/auditbeat", "logs": "/var/log/auditbeat"}, "type": "auditbeat", "uuid": "67b75f47-b666-4b1c-beae-dde90079e8f1"}}}
2019-08-09T02:04:41.390Z	INFO	[beat]	instance/beat.go:911	Build info	{"system_info": {"build": {"commit": "6f0ec01a0e57fe7d4fd703b017fb5a2f6448d097", "libbeat": "7.3.0", "time": "2019-07-24T17:38:3

### Add template from Auditbeat to elasticsearch

In [42]:
%%bash

sudo auditbeat setup --index-management \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=["<ELASTICSEARCH-IP-ADDRESS>:9200"] \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem


Index setup finished.


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### List Loaded Templates

In [43]:
%%bash
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/' | jq 'keys'

[
  ".data-frame-internal-1",
  ".data-frame-notifications-1",
  ".kibana_task_manager",
  ".logstash-management",
  ".management-beats",
  ".ml-anomalies-",
  ".ml-config",
  ".ml-meta",
  ".ml-notifications",
  ".ml-state",
  ".monitoring-alerts-7",
  ".monitoring-beats",
  ".monitoring-es",
  ".monitoring-kibana",
  ".monitoring-logstash",
  ".triggered_watches",
  ".watch-history-10",
  ".watches",
  "auditbeat-7.3.0",
  "filebeat-7.3.0",
  "metricbeat-7.3.0",
  "packetbeat-7.3.0"
]


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  330k  100  330k    0     0  6357k      0 --:--:-- --:--:-- --:--:-- 6357k


### Start Auditbeat

In [44]:
%%bash
sudo systemctl start auditbeat
sudo systemctl enable auditbeat

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Synchronizing state of auditbeat.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable auditbeat
sh: 0: getcwd() failed: No such file or directory


### Alternatively, you can run Metricbeat in debug mode by using the following command:

~~~
sudo /usr/share/auditbeat/bin/auditbeat -e -d "publish" -c /etc/auditbeat/auditbeat.yml
~~~

### Heartbeat Installation

In [45]:
%%bash
sudo apt install -y heartbeat-elastic

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  grub-pc-bin
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  heartbeat-elastic
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 19.2 MB of archives.
After this operation, 68.9 MB of additional disk space will be used.
Get:1 https://artifacts.elastic.co/packages/7.x/apt stable/main amd64 heartbeat-elastic amd64 7.3.0 [19.2 MB]
Fetched 19.2 MB in 2s (10.7 MB/s)
Selecting previously unselected package heartbeat-elastic.
(Reading database ... 225385 files and directories currently installed.)
Preparing to unpack .../heartbeat-elastic_7.3.0_amd64.deb ...
Unpacking heartbeat-elastic (7.3.0) ...
Processing triggers for ureadahead (0.100.0-21) ...
Setting up heartbeat-elastic (7.3.0) ...
Processing triggers for systemd (237-3ubuntu10.25) ...
Processing triggers for ureada

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
dpkg-preconfigure: unable to re-open stdin: No such file or directory
sh: 0: getcwd() failed: No such file or directory


### Disable default configuration

##### The default heartbeat configuration enables connectivity directly to elasticseach. We will disable this configuration to later enable communication to logstash.

In [46]:
%%bash
sudo sed -i 's/^output.elasticsearch\:/#&/' /etc/heartbeat/heartbeat.yml
sudo sed -i 's/^\ \ hosts\: \[\"localhost\:9200\"\]/#&/' /etc/heartbeat/heartbeat.yml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enable communication with logstash

In [47]:
%%bash
export ls_ip="<LOGSTASH-IP-ADDRESS>"

#Enable Logstash output
sudo sed -i '/output.logstash\:/s/^#//g' /etc/heartbeat/heartbeat.yml

#Uncomment and Change Logstash IP ADDRESS
sudo sed -i "s/#hosts\: \[\"localhost\:5044\"\]/hosts\: \[\"$ls_ip\:5044\"\]/g" /etc/heartbeat/heartbeat.yml
sudo sed -i 's/#ssl\.certificate_authorities\: \["\/etc\/pki\/root\/ca\.pem"\]/ssl\.certificate\_authorities\: \["\/etc\/pki\/tls\/certs\/logstash\-forwarder\.crt"\]/g' /etc/heartbeat/heartbeat.yml


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Enable Monitoring your Target Website

In [48]:
%%bash
export kb_ip="<TARGET-IP-ADDRESS>"
sudo sed -i "s/urls\: \[\"http\:\/\/localhost\:9200\"\]/urls\: \[\"http\:\/\/$kb_ip\"]/g" /etc/heartbeat/heartbeat.yml

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Test Configuration and Connectivity to Logstash

In [49]:
%%bash
sudo /usr/share/heartbeat/bin/heartbeat test config -c /etc/heartbeat/heartbeat.yml
sudo /usr/share/heartbeat/bin/heartbeat test output -c /etc/heartbeat/heartbeat.yml

Config OK
logstash: 10.1.1.21:5044...
  connection...
    parse host... OK
    dns lookup... OK
    addresses: 10.1.1.21
    dial up... OK
  TLS...
    security: server's certificate chain verification is enabled
    handshake... OK
    TLS version: TLSv1.2
    dial up... OK
  talk to server... OK


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### Add Kibana Dashboards for Heartbeat

##### The original file was collected from the following page:

~~~
https://github.com/elastic/uptime-contrib
~~~

##### Open the file named "http_dashboard.json" in RAW in the following link:

https://github.com/elastic/uptime-contrib/blob/master/dashboards/http_dashboard.json

##### Copy all the contents and paste them into your favorite test editor and save it as "http_dashboard.json" 

##### Upload your saved file by going to Management --> Saved objects --> Import in Kibana

### Add template from auditbeat to elasticsearch

In [50]:
%%bash

sudo heartbeat setup --index-management \
  -E output.logstash.enabled=false \
  -E output.elasticsearch.hosts=["<ELASTICSEARCH-IP-ADDRESS>:9200"] \
  -E output.elasticsearch.username=elastic \
  -E output.elasticsearch.password=elasticsiem


Index setup finished.


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


### List Loaded Templates

In [51]:
%%bash
curl -u elastic:elasticsiem -XGET '<ELASTICSEARCH-IP-ADDRESS>:9200/_template/' | jq 'keys'

[
  ".data-frame-internal-1",
  ".data-frame-notifications-1",
  ".kibana_task_manager",
  ".logstash-management",
  ".management-beats",
  ".ml-anomalies-",
  ".ml-config",
  ".ml-meta",
  ".ml-notifications",
  ".ml-state",
  ".monitoring-alerts-7",
  ".monitoring-beats",
  ".monitoring-es",
  ".monitoring-kibana",
  ".monitoring-logstash",
  ".triggered_watches",
  ".watch-history-10",
  ".watches",
  "auditbeat-7.3.0",
  "filebeat-7.3.0",
  "heartbeat-7.3.0",
  "metricbeat-7.3.0",
  "packetbeat-7.3.0"
]


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  352k  100  352k    0     0  6188k      0 --:--:-- --:--:-- --:--:-- 6188k


### Start Heartbeat

In [52]:
%%bash
sudo systemctl start heartbeat-elastic
sudo systemctl enable heartbeat-elastic

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Synchronizing state of heartbeat-elastic.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable heartbeat-elastic
sh: 0: getcwd() failed: No such file or directory


### Alternatively, you can run Heartbeat in debug mode by using the following command:

~~~
sudo /usr/share/hearteat/bin/heartbeat -e -d "publish" -c /etc/heartbeat/heartbeat.yml
~~~